In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from scipy.stats import norm
#from scipy.stats import linregress
#import statsmodels.formula.api as smf
import csv
import scipy.optimize as opt
import pandas_datareader as pdr
from datetime import datetime


In [2]:
# function to read in data from a csv
def readPricesCSV(file):
    df = pd.read_csv(file)
    dfPrices = df["Adj Close"]
    return dfPrices

In [3]:
# function to read in stock prices from yahoo finance
def readPricesYF(start_date, end_date, tickers):
    stockPrices = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)   # read in the data
    stockPrices = stockPrices.filter(like='Adj Close')   # tickers are columns and data values are adjusted closing prices with the date as the index
    stockPrices.columns = tickers   # change column names to be their tickers
    stockPrices = stockPrices.stack().swaplevel().sort_index().reset_index()    # make the data tall and clean it up so it is easier to perform analysis on
    stockPrices.columns = ['Firm','Date','Adj Close']     # rename the columns
#    stockPrices['Return'] = stockPrices.groupby('Firm')['Adj Close'].pct_change()    # get the daily returns for each ticker
    return stockPrices


In [4]:
# the objective function
def obj(weights):
    
    sum_return = np.dot(weights,means)     #get the return measure given by the weights and mean
    num_stocks = len(means)
    
    #get the risk measure given by the weights
    risk_measure = np.matmul(np.matmul(np.transpose(np.array(weights)), cov_matr), np.array(weights))

    #return the objective function value
    return -(sum_return - (risk_level * risk_measure))


In [5]:
#choose which stocks to include in the optimization of the portfolio (list the tickers)
tickers = ['MSFT', 'AAPL', 'AMZN', 'NFLX']
tickers = sorted(tickers)

#set the start and end dates- choose last six months in this case
start_date = datetime(2020, 8, 25) #year, month, day
end_date = datetime(2021,2,25) #year, month, day

stockPrices = readPricesYF(start_date, end_date, tickers)    # use the function defined above to get data from Yahoo Finance


In [6]:
stockPrices

,Firm,Date,Adj Close
0,AAPL,2020-08-25,124.424088
1,AAPL,2020-08-26,126.116135
2,AAPL,2020-08-27,124.608498
3,AAPL,2020-08-28,124.406647
4,AAPL,2020-08-31,128.625549
...,...,...,...
503,NFLX,2021-02-19,540.219971
504,NFLX,2021-02-22,533.780029
505,NFLX,2021-02-23,546.150024
506,NFLX,2021-02-24,553.409973


In [7]:
# reformat the data
stock_prices_format = stockPrices.set_index(['Date', 'Firm']).unstack()   # change format so that date and each company are columns
stock_prices_format.reset_index()

Date   Adj Close                                     
Firm                   AAPL         AMZN        MSFT        NFLX
0    2020-08-25  124.424088  3346.489990  215.408630  490.579987
1    2020-08-26  126.116135  3441.850098  220.065674  547.530029
2    2020-08-27  124.608498  3400.000000  225.469070  526.270020
3    2020-08-28  124.406647  3401.800049  227.787643  523.890015
4    2020-08-31  128.625549  3450.959961  224.424194  529.559998
..          ...         ...          ...         ...         ...
122  2021-02-19  129.869995  3249.899902  240.970001  540.219971
123  2021-02-22  126.000000  3180.739990  234.509995  533.780029
124  2021-02-23  125.860001  3194.500000  233.270004  546.150024
125  2021-02-24  125.349998  3159.530029  234.550003  553.409973
126  2021-02-25  120.989998  3057.159912  228.990005  546.700012

[127 rows x 5 columns]

In [8]:
# get the different number of stocks and the number of stock prices 
num_stocks = len(tickers)   # the number of stocks
prices_per_stock = stock_prices_format.shape[0]   # the number of stock prices

In [9]:
# get the covariance matrix
cov_matr = np.cov(np.array(stock_prices_format.iloc[:,range(0,num_stocks)].T))   # get the columns and transpose it so it is in right format, then turn it into covariance matrix
cov_matr

array([[   78.14430733,   558.11492864,    84.26230921,   180.56013459],
       [  558.11492864, 12168.77726018,   810.26625573,  2169.09772824],
       [   84.26230921,   810.26625573,   136.02137507,   251.58777063],
       [  180.56013459,  2169.09772824,   251.58777063,   764.99314083]])

In [10]:
# get the mean price for each stock

means = np.array(stock_prices_format.mean().to_list())    # get means for each column, convert to a list, convert to a numpy array
means

array([ 122.99478972, 3207.85959223,  218.25363448,  514.73921264])

In [11]:
# set the risk level
risk_level = 1   # risk loving < 0; risk neutral = 0; risk averse > 0

In [12]:
# set an intial value for the weights
weights = np.array([0.25,0.25,0.25,0.25]) # set an initial value for the weights

In [13]:
# perform the optimization!
lin_constr = opt.LinearConstraint([1,1,1,1], [1], [1])    # sum of all values are greater than or equal to 1 and less than or equal to 1, so the sum has to be equal to 1
bounds = opt.Bounds([0]*num_stocks, [0.5]*num_stocks)    # each portfolio weight is greater than 0 and less than 0.5
result = opt.minimize(obj, x0=[0.25]*num_stocks, method="trust-constr", constraints = lin_constr, bounds=bounds)    # actually perform the optimization
#print(result.fun)
#print(result.x)

optimal_weights = pd.DataFrame({'stock': tickers, 'weights': result.x.tolist()})
optimal_weights
#sum(optimal_weights['weights'])

,stock,weights
0,AAPL,4.660590e-01
1,AMZN,8.554193e-02
2,MSFT,4.483989e-01
3,NFLX,2.473065e-07
